# Taller #24: Interfaces Multimodales: Uniendo Voz y Gestos
#### Desarrollado por: David Santiago Cruz Hernández

In [1]:
import speech_recognition as sr
import tkinter as tk
from pythonosc import udp_client
import threading
import cv2
import mediapipe as mp

# pip install -r requirements.txt


### Configuración del cliente OSC

In [2]:
IP_DESTINO = "127.0.0.1"  # IP del servidor (Processing)
PUERTO_OSC = 9124         # Puerto OSC
cliente_osc = udp_client.SimpleUDPClient(IP_DESTINO, PUERTO_OSC)
detener_hilo = threading.Event()

### Lista de comandos permitidos

In [3]:
COMANDOS_VOZ = {'iniciar','cambiar color', 'mover', 'transformar', 'detener'}
GESTOS_VALIDOS = {'mano_abierta', 'mano_cerrada', 'un_dedo', 'dos_dedos', 'tres_dedos', 'otro'}

### Estados actuales

In [4]:
estado_voz = None
estado_gesto = None

### Interfaz gráfica con Tkinter

In [5]:
class AppVisual(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Control por Voz y gestos")
        self.geometry("500x400")
        self.color_actual = "white"
        self.pos_x = 200
        self.label = tk.Label(self, text="Esperando comando...", bg=self.color_actual, width=30, height=10, font=("Arial", 20))
        self.label.place(relx=0.5, rely=0.5, anchor="center")


    def actualizar_interfaz(self):
        global estado_voz, estado_gesto
        if estado_voz == "iniciar" and estado_gesto == "un_dedo":
            self.label.config(text="Iniciando...")
            cliente_osc.send_message("/accion", ["iniciar_animacion"])
            estado_voz = None
        elif estado_voz == "cambiar color" and estado_gesto == "mano_abierta":
            self.label.config(bg=self.color_actual)
            cliente_osc.send_message("/accion", ["cambiar_color"])
            estado_voz = None
        elif estado_voz == "mover" and estado_gesto == "dos_dedos":
            self.label.config(text="Moviendo...")
            cliente_osc.send_message("/accion", ["mover_objeto"])
            estado_voz = None
        elif estado_voz == "transformar" and estado_gesto == "tres_dedos":
            self.label.config(text="Transformando...")
            cliente_osc.send_message("/accion", ["cambiar_forma"])
            estado_voz = None
        elif estado_voz == "detener" and estado_gesto == "mano_cerrada":
            self.label.config(text="Deteniendo...")
            cliente_osc.send_message("/accion", ["detener_animacion"])
            estado_voz = None
        self.after(100, self.actualizar_interfaz)

    def actualizar_estado_voz(self, texto):
        self.label.config(text=texto.capitalize())

### Función de reconocimiento de voz

In [6]:
def escuchar_voz(app):
    r = sr.Recognizer()
    with sr.Microphone() as source:
        while True:
            try:
                audio = r.listen(source, timeout=5, phrase_time_limit=3)
                texto = r.recognize_google(audio, language="es-CO").lower()
                if texto in COMANDOS_VOZ:
                    print(f"[VOZ] Comando reconocido: {texto}")
                    global estado_voz
                    estado_voz = texto
                # Programar actualización en el hilo principal de Tkinter
                app.after(0, lambda: app.actualizar_estado_voz(texto))
            except sr.UnknownValueError:
                pass
            except Exception as e:
                #print(f"[ERROR VOZ]: {e}")
                pass

### Función de detección de gestos

In [8]:
def detectar_gesto_mano(hand_landmarks, mp_hands):
    # Obtiene las posiciones de las puntas y nudillos de los dedos
    dedos = {
        "PULGAR": (mp_hands.HandLandmark.THUMB_TIP, mp_hands.HandLandmark.THUMB_IP),
        "INDICE": (mp_hands.HandLandmark.INDEX_FINGER_TIP, mp_hands.HandLandmark.INDEX_FINGER_PIP),
        "MEDIO": (mp_hands.HandLandmark.MIDDLE_FINGER_TIP, mp_hands.HandLandmark.MIDDLE_FINGER_PIP),
        "ANULAR": (mp_hands.HandLandmark.RING_FINGER_TIP, mp_hands.HandLandmark.RING_FINGER_PIP),
        "MENIQUE": (mp_hands.HandLandmark.PINKY_TIP, mp_hands.HandLandmark.PINKY_PIP),
    }
    extendidos = {}
    for nombre, (tip, pip) in dedos.items():
        extendidos[nombre] = hand_landmarks.landmark[tip].y < hand_landmarks.landmark[pip].y

    # Lógica para los gestos
    if all(extendidos.values()):
        return "mano_abierta"
    elif not any(extendidos.values()):
        return "mano_cerrada"
    elif extendidos["INDICE"] and not extendidos["MENIQUE"] and not extendidos["PULGAR"] and not extendidos["ANULAR"] and not extendidos["MEDIO"]:
        return "un_dedo"
    elif extendidos["PULGAR"] and extendidos["MENIQUE"] and not extendidos["INDICE"] and not extendidos["ANULAR"] and not extendidos["MEDIO"]:
        return "dos_dedos"
    elif extendidos["PULGAR"] and extendidos["INDICE"] and extendidos["MENIQUE"] and not extendidos["MEDIO"] and not extendidos["ANULAR"]:
        return "tres_dedos"
    return "otro"

In [9]:
def detectar_gestos():
    global estado_gesto
    mp_hands = mp.solutions.hands
    mp_dibujo = mp.solutions.drawing_utils
    hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(rgb_frame)

        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                # Dibuja los puntos y líneas sobre la mano
                mp_dibujo.draw_landmarks(
                    frame, hand_landmarks, mp_hands.HAND_CONNECTIONS
                )
                
                # Detecta el gesto de la mano
                estado_gesto = detectar_gesto_mano(hand_landmarks, mp_hands)
        else:
            estado_gesto = None

        cv2.imshow('Gestos de Mano', frame)
        if cv2.waitKey(1) == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

### Actualización periódica de la interfaz

### Ejecutar e Iniciar hilos

In [ ]:
app = AppVisual()
threading.Thread(target=escuchar_voz, args=(app,), daemon=True).start()
threading.Thread(target=detectar_gestos, daemon=True).start()
app.after(100, app.actualizar_interfaz)
app.mainloop()

[VOZ] Comando reconocido: iniciar
[VOZ] Comando reconocido: iniciar
[VOZ] Comando reconocido: mover
[VOZ] Comando reconocido: transformar
[VOZ] Comando reconocido: transformar
[VOZ] Comando reconocido: cambiar color
[VOZ] Comando reconocido: detener
